In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [6]:
labels = gpd.read_file('data/processed-labels/seattle.shp')

In [73]:
labels

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860)
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.33189 47.61781)
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751)
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752)
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077)
195539,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344 47.55229)
195540,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327)
195541,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347)


In [16]:
#read in all files in the feature folder
clustered = pd.read_csv('data/features/clustered.csv')
fp = pd.read_csv('data/features/false_positives.csv')
distance = pd.read_csv('data/features/distance.csv')
comments = pd.read_csv('data/_archive/labels.csv')

In [17]:
clustered

,label_id,clustered,count
0,9,1.0,6.0
1,10,1.0,6.0
2,11,0.0,10.0
3,12,0.0,3.0
4,13,0.0,9.0
...,...,...,...
195538,116833,0.0,0.0
195539,37995,0.0,0.0
195540,37983,0.0,0.0
195541,37989,0.0,0.0


In [18]:
#select label_id, label_tags, label_description
comments = comments[['label_id', 'label_tags', 'label_description']]

In [19]:
#delete duplicates
comments = comments.drop_duplicates(subset=['label_id'])

In [20]:
comments

,label_id,label_tags,label_description
0,85055,0.0,0.0
1,85057,0.0,0.0
2,85059,0.0,0.0
3,85060,0.0,0.0
4,85062,0.0,0.0
...,...,...,...
196809,231270,0.0,0.0
196810,231285,0.0,0.0
196811,231338,0.0,0.0
196812,231339,0.0,0.0


In [21]:
#merge all into one on label_id
df = pd.merge(labels, clustered, on='label_id')
df = pd.merge(df, fp, on='label_id')
df = pd.merge(df, distance, on='label_id')
df = pd.merge(df, comments, on='label_id', how='inner')

In [22]:
df

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,clustered,count,false_positive,distance,label_tags,label_description
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860),0.0,1.0,0,5.435411,0.0,0.0
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.33189 47.61781),0.0,7.0,0,12.187807,0.0,0.0
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751),1.0,6.0,0,1.261442,0.0,0.0
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752),0.0,4.0,0,0.859638,0.0,0.0
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764),0.0,6.0,0,2.443982,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077),1.0,10.0,0,19.281464,0.0,0.0
195539,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344 47.55229),1.0,12.0,0,7.199212,0.0,0.0
195540,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327),0.0,9.0,1,14.445776,0.0,0.0
195541,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347),0.0,9.0,1,20.171446,0.0,0.0


In [23]:
gt = pd.read_csv('data/seattle_ground_truth_labels.csv')

In [24]:
#merge gt with labels how left
b= pd.merge(df, gt, on='label_id', how='right')

In [25]:
#rename ground truth to verified
b.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [26]:
#drop the geometry column and save as csv
b.drop(columns=['geometry']).to_csv('data/test_set_seattle.csv', index=False)

In [27]:
b.verified.value_counts()

1    12867
0     3713
Name: verified, dtype: int64

In [28]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(gt.label_id)]

In [66]:
#save the training set into a shapefile
training.to_file('data/training_set_seattle.shp')

In [29]:
#drop the geometry column and save as csv
training.drop(columns=['geometry']).to_csv('data/training_set_seattle.csv', index=False)

In [12]:
195543-16580

178963